<a href="https://colab.research.google.com/github/Iispar/dl-in-hlt-project/blob/main/course_project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learning in Human Language Technology Project (Template)

- Student(s) Name(s): Iiro Partanen
- Date: 18/10/2023
- Chosen Corpus: amazon_reviews_multi
- Contributions (if group project): -

### Corpus information

- Description of the chosen corpus: includes 200k of amazon reviews in English, Japanese, German, French, Chinese and Spanish.
- Paper(s) and other published materials related to the corpus: Paper from the courpus is "The Multilingual Amazon Reviews Corpus"
- Random baseline performance and expected performance for recent machine learned models:random baseline performance will be about 20% and the performance in the paper is 59% lets aim for this.

---

## 1. Setup

In [1]:
!pip3 install -q transformers datasets evaluate
!pip install optuna
!pip install accelerate -U
from transformers import DistilBertTokenizer
from transformers import BertTokenizer, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import DistilBertModel, BertModel
from transformers import AutoModelForSequenceClassification
import datasets
import sklearn.feature_extraction
import torch
import transformers
import numpy as np
import evaluate
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
dset = 'mteb/amazon_reviews_multi'
model = 'bert-base-multilingual-cased'

---

## 2. Data download, sampling and preprocessing

### 2.1. Download the corpus

In [3]:
from datasets.combine import concatenate_datasets
engDataset = datasets.load_dataset(dset, name='en'); # imports the dataset. eng
deDataset = datasets.load_dataset(dset, name='de'); # imports the dataset. de
# check it works
print(engDataset);
combinedDset = concatenate_datasets([engDataset['train'], deDataset['train']]) # combine train
combinedTest = concatenate_datasets([engDataset['test'], deDataset['test']]) # combine test
print(combinedDset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})
Dataset({
    features: ['id', 'text', 'label', 'label_text'],
    num_rows: 400000
})


In [ ]:
#FOR TESTING
# engDataset["train"] = engDataset["train"].select(range(100000))

In [4]:
print(engDataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 5000
    })
})


### 2.2. Sampling and preprocessing

In [5]:
engDataset = engDataset.shuffle() # shuffle the dataset for safety.
engDataset = engDataset.remove_columns(['id', 'label_text']) # removes everything that we don't need

combinedDset = combinedDset.shuffle();
combinedDset = combinedDset.remove_columns(['id', 'label_text'])

combinedTest = combinedTest.shuffle();
combinedTest = combinedTest.remove_columns(['id', 'label_text'])

deDataset = deDataset.shuffle();
deDataset = deDataset.remove_columns(['id', 'label_text'])

In [6]:
# lets look at five results to see if there is more preprocessing to be done

print(engDataset['train'][0]['text'])

# looks like the title is spaced with \n\n, but other than that there is no problems. Looks good to me.

Great Bathroom Organizer

I installed the shelves In my client’s bathroom to remove the clutter around their tubs. It worked perfectly.


### tokenization

In [7]:
# Load the BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

In [8]:
# tokenizes one example
def tokenize_example(example):
    split = example['text'].split('\n\n'); # splits the sentace and title.
    return tokenizer.encode_plus(split[0], split[1],
             truncation='only_second',
             add_special_tokens=True,
             return_attention_mask=True,
             return_overflowing_tokens=False,
             return_special_tokens_mask=False,
             max_length=512,
             pad_to_max_length=False)

In [9]:
# map the whole dsets

eng_tokenized = engDataset.map(tokenize_example)
combined_tokenized = combinedDset.map(tokenize_example)
combinedTest_tokenized = combinedTest.map(tokenize_example)
de_tokenized = deDataset.map(tokenize_example)

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/400000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [10]:
print(eng_tokenized['train'][1])
print(tokenizer.decode(eng_tokenized['train'][1]['input_ids']))

# looks good to me.

{'text': 'Cute fit\n\nI got this in two colors, black and light blue. I absolutely LOVE the black one! Relatively good quality, fits well, super cute. The blue one however came with a bit of discoloration and some threads loose.', 'label': 2, 'input_ids': [101, 51275, 10112, 21635, 102, 146, 19556, 10531, 10106, 10551, 36888, 117, 15045, 10111, 15765, 23254, 119, 146, 48573, 10454, 52734, 10105, 15045, 10464, 106, 20304, 95155, 10454, 15198, 21905, 117, 21635, 10107, 11206, 117, 25212, 21610, 10112, 119, 10117, 23254, 10464, 13800, 13383, 10169, 169, 17684, 10108, 14884, 61465, 10822, 10111, 11152, 77586, 66058, 10107, 102920, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

---

## 3. Machine learning model

### 3.1. Model training

In [11]:
# config
%%time
import torch
import torch.nn as nn

# Create the BertClassfier class
class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()
        # hidden size of BERT (always 768), hidden size of our classifier, and number of labels (in this case 5)
        D_in, H, D_out = 768, 25, 5

        # load the pretrained bert.
        self.bert = BertModel.from_pretrained(model)

        # basic one layer feed forward network that outputs the labels.
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Linear(H, D_out)
        )
    def forward(self, input_ids, attention_mask, labels=None):
        # run the BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)

        # Extract the last hidden state of the token for classification
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed tha last hidden state into the classifier. This outputs the labels.
        logits = self.classifier(last_hidden_state_cls)

        if labels is not None:
          # calculates the loss.
          loss = torch.nn.CrossEntropyLoss();
          return (loss(logits,labels),logits);
        else:
          # if no labels, just return the logits
          return (logits,);
       # torch.cuda.empty_cache();


from transformers import AdamW, get_linear_schedule_with_warmup


CPU times: user 137 µs, sys: 0 ns, total: 137 µs
Wall time: 142 µs


In [12]:
accuracy = evaluate.load('accuracy');
def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels;
    predictions = np.argmax(outputs, axis=-1); #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels); # calc accuracy

In [13]:
model = BertClassifier()

In [14]:
# Training params. We optimize these later
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy = 'steps',
    logging_strategy = 'steps',
    eval_steps = 500,
    logging_steps = 500,
    learning_rate=1.2668184349858087e-05,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    max_steps = 20000,
    num_train_epochs=4,
    weight_decay=0.01,
  )

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
early_stopping = transformers.EarlyStoppingCallback(3); # stop training if the eval loss is not getting better.

# Set the trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = eng_tokenized['train'],
    eval_dataset = eng_tokenized['test'],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_accuracy,
)

# train the model
trainer.train()


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,1.352400,1.202393,0.513600
1000,1.116500,1.041490,0.571800
1500,1.034400,1.023288,0.570000
2000,1.034700,0.992876,0.573600
2500,0.976200,0.943909,0.592400
3000,0.957900,0.969377,0.590000
3500,0.940900,1.020122,0.586800
4000,0.928700,0.934466,0.599400
4500,0.927000,0.979074,0.591200
5000,0.918000,0.929940,0.599800


KeyboardInterrupt: ignored

### 3.2 Hyperparameter optimization

# HOX
A little about this this is done over multiple days as training one model takes a bit of time so you wont see all the results but the best one currently is:

lr: 1.2668184349858087e-05
batch: 8
steps: 500
epoch: 4
max steps: 20k ( ab 1hr train time with 16 batch)

In [ ]:
# Used optuna for optimization

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 4, 16])
    epochs=trial.suggest_int('num_train_epochs', low = 2,high = 6),

    # params
    trainer_args = transformers.TrainingArguments(
        "mlp_checkpoints",
        evaluation_strategy = "steps",
        logging_strategy = "steps",
        eval_steps = 500,
        logging_steps = 500,
        learning_rate = learning_rate,
        max_steps = 20000,
        load_best_model_at_end = True,
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
        num_train_epochs = epochs
    )

    # the model
    mlp = model
    early_stopping = transformers.EarlyStoppingCallback(3); # stop training if the eval loss is not getting better.

    # train a model
    trainer = transformers.Trainer(
        model = mlp,
        args = trainer_args,
        train_dataset = eng_tokenized['train'],
        eval_dataset = eng_tokenized['test'],
        compute_metrics = compute_accuracy,
        data_collator = data_collator,
        callbacks = [early_stopping]
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_accuracy"] # return the best result.

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

### 3.3. Evaluation on test set

In [ ]:
# Your code to evaluate the final model on the test set here
eval_results = trainer.evaluate(eng_tokenized["validation"])

print(eval_results)

Step,Training Loss,Validation Loss,Accuracy
500,1.252400,1.130167,0.519600
1000,1.074600,1.034022,0.562600
1500,1.030800,0.970599,0.593400
2000,1.014900,0.981329,0.579400
2500,0.975000,0.962946,0.585200
3000,0.944400,0.940313,0.593800
3500,0.908200,0.916086,0.604000
4000,0.934400,0.949661,0.598600
4500,0.935900,0.909991,0.611400
5000,0.894000,0.957910,0.585800


{'eval_loss': 0.8872833847999573, 'eval_accuracy': 0.6218}


got 0.62

### 3.4. Multilingual and cross-lingual experiments

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = combined_tokenized, # train with combined
    eval_dataset = combinedTest_tokenized,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_accuracy,
)

# train the model
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.996700,0.984961,0.574800
1000,0.972000,0.955562,0.589500
1500,0.964100,0.982700,0.553000
2000,0.957300,0.921952,0.600000
2500,0.940800,0.925412,0.608000
3000,0.959200,0.898878,0.612500
3500,0.933600,0.890425,0.614300
4000,0.909100,0.890920,0.615900
4500,0.906600,0.889673,0.615900
5000,0.891600,0.929874,0.596000


KeyboardInterrupt: ignored

In [ ]:
eval_results = trainer.evaluate(eng_tokenized["validation"])

print(eval_results)

Step,Training Loss,Validation Loss,Accuracy
500,0.996700,0.984961,0.574800
1000,0.972000,0.955562,0.589500
1500,0.964100,0.982700,0.553000
2000,0.957300,0.921952,0.600000
2500,0.940800,0.925412,0.608000
3000,0.959200,0.898878,0.612500
3500,0.933600,0.890425,0.614300
4000,0.909100,0.890920,0.615900
4500,0.906600,0.889673,0.615900
5000,0.891600,0.929874,0.596000


{'eval_loss': 0.8668937683105469, 'eval_accuracy': 0.6228}


Got 0.62

In [15]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = de_tokenized['train'], # train with de
    eval_dataset = de_tokenized['test'],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_accuracy,
)

# train the model
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,1.100900,1.007103,0.560000
1000,1.004400,0.947415,0.600400
1500,1.018800,0.940233,0.583400
2000,0.974400,0.917034,0.608400
2500,0.960500,0.922702,0.607400
3000,0.958000,0.941678,0.591000
3500,0.945700,0.894049,0.613600
4000,0.947600,0.886063,0.615000
4500,0.925600,0.891841,0.603800
5000,0.924300,0.909235,0.610600


RuntimeError: ignored

In [23]:
res = trainer.evaluate(eng_tokenized["validation"])

print(res)

Step,Training Loss,Validation Loss,Accuracy
500,1.100900,1.007103,0.560000
1000,1.004400,0.947415,0.600400
1500,1.018800,0.940233,0.583400
2000,0.974400,0.917034,0.608400
2500,0.960500,0.922702,0.607400
3000,0.958000,0.941678,0.591000
3500,0.945700,0.894049,0.613600
4000,0.947600,0.886063,0.615000
4500,0.925600,0.891841,0.603800
5000,0.924300,0.909235,0.610600


{'eval_loss': 0.9017043113708496, 'eval_accuracy': 0.612}


got 0.61 (from 0.64)

---

## 4. Results and summary

### 4.1 Corpus insights

The corpus has six languages split into 20% splits and all of them annotated witht the stars of the review. Almost all the reviews are from different items and reviewers. Reviews are in different lengths but exluding Chinese the average length of the review and title are similar lengths.

### 4.2 Results

The 62% I achieved on the English dataset is not bad as it is the same as the paper had with the whole dataset. In the eng with a multilingual model I am happy with that when it is just a single language. I had done this same task on a base bert and achieved over 70% which shows the difference between multilingual and base bert.

With the two language model that was being evaluated on English it didn't perform any better than the original model which was expectable as the multilingual model is a bit biased to English as in it has a lot more data in English than the other languages so it will not benefit from the other languages.

Lastly the trained on other language and evaluated on english. The training wen't abit further than others (oops..), but we can see that the evalueation gave result 0.61 when the training on the language reached 0.64. So in the end it resulted in 0.03 worse results. This is still not bad as almost reaching the same that the English model reached on less training.


### 4.3 Relation to random baseline / expected performance / state of the art

State of the art performance seems to be in accuracy at 90%. I don't know how true this is but the Huggingface leaderboards seem to say this. So didn't really achieve this but achieving the papers 59% with the only English model is in my opinion a good result.

In the end I could have reached better results by training longer, but because of Colab this was not possible.

## 5. Bonus Task (optional)

### 5.1. Data selection

(Briefly describe how many English and target language examples were used and how these were selected, include relevant code)

### 5.2 Sentence representations

In [ ]:
# Your code to create a sentence embedding for the given text here

### 5.3. Cosine similarity

In [ ]:
# Your code to calculate the cosine similarity of the embeddings and select the target sentence that maximizes the cosine similarity here

### 5.4 Bonus task evaluation

(Present the evaluation results here)